In [1]:
import pke

from pathlib import Path
import glob

from nltk.corpus import stopwords

### Run YAKE Baseline one Inspec Dataset

In [3]:
# Get id of the test documents from the benchmarking path (../ake-datasets/../test)
# Modify the path (i.e., redirect to the ground-truth folder) to geth the gold standard keywords
test_docs= glob.glob('../ake-datasets/datasets/Inspec/test/*.xml')
test_docs= ['../Inspec/docsutf8/'+doc.split('/')[-1][:-3]+'txt' for doc in test_docs]

Path("./Output/TextRank-Output/").mkdir(parents=True, exist_ok=True)

In [6]:
# initialize keyphrase extraction model, here TextRank
extractor = pke.unsupervised.TextRank()
stoplist = stopwords.words('english')

window = 2
use_stems = False # use stems instead of words for weighting
threshold = 0.8

for file in test_docs:     
    
    input_text= open(file, mode='r').read()
    input_text=input_text.replace('\t', ' ').replace('\n', '')
    
    extractor.load_document(input=input_text, language='en')

    extractor.candidate_selection(n=3, stoplist=stoplist)    
    extractor.candidate_weighting(window=window, stoplist=stoplist, use_stems=use_stems)

    keyphrases = extractor.get_n_best(n=10, threshold=threshold)
   
    keyphrases_list= '\n'.join([key[0] for key in keyphrases])
    
    with open('./Output/TextRank-Output/'+file.split('/')[-1], 'w') as outFile:
        outFile.writelines(keyphrases_list)
        
    outFile.close()